# Determine Date Ranges across Clusters and Assign Range back to Cluster or Unique Fire Table:

In [26]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from geopy.distance import great_circle
import math
from scipy import spatial

## Loading data files and reducing Emissions Data to a year by year dataframe. 

In [2]:
pd.set_option('display.max_columns', 40)

FiresClusters2010 = pd.read_csv('/Users/AlfHaugen/Python/Wildfire_Data/FireExports/Emissions Cluster Data/Emissions2010_DBScan_Clusters.csv')

In [4]:
Clusterpoints2010 = pd.read_csv('/Users/AlfHaugen/Python/Wildfire_Data/FireExports/Emissions Cluster Data/Emclustermakeup_2010.csv')

In [7]:
emdata = pd.read_csv('/Users/AlfHaugen/Python/Wildfire_Data/9 Missoula Emisions Data RDS-2017-0039/Emissions_Year/emissions_year2003to2015.csv')

## Pairing down Emissions Data to just the year that matches the Cluster DB Scan year. 

## Select/Mark Year for Run

In [38]:
pullyear = 2010

In [39]:
emdata2010 = emdata[(emdata.year == pullyear)]
emdata2010.head()

,id,year,doy,longitude,latitude,grid10k,covertype,fuelcode,area_burned,prefire_fuel,consumed_fuel,ECO2,ECO,ECH4,EPM2.5,cwd_frac,duff_frac,fuel_moisture_class,burn_source,burnday_source,BSEV,BSEV_flag
3865595,0,2010,335,-80.5883,25.4635,5923,95,0,62500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,77,2,0
3865596,1,2010,335,-80.5858,25.4631,5923,95,0,62500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,77,2,0
3865597,2,2010,335,-80.5834,25.4628,5923,95,0,62500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,77,2,0
3865598,3,2010,335,-80.5810,25.4624,5923,95,0,62500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,77,2,0
3865599,4,2010,335,-80.5879,25.4657,5923,95,0,62500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,77,2,0


In [40]:
Clusterpoints2010.shape

(270486, 5)

In [41]:
Clusterpoints2010.tail(3)

,Unnamed: 0,0,1,ClusterNum,Year
270483,270483,48.7412,-119.5757,8234,2010
270484,270484,48.8160,-119.2463,8235,2010
270485,270485,48.9655,-119.2912,8236,2010


### Testing iloc function across cluster points DF:
Clusterpoints2011.iloc[710000]

In [42]:
clusterindex = 2
clusterindex = int(clusterindex)

templat = Clusterpoints2010.iloc[clusterindex]['0']
templong = Clusterpoints2010.iloc[clusterindex]['1']
tempyear = Clusterpoints2010.iloc[clusterindex]['Year']
tempCNmax = Clusterpoints2010['ClusterNum'].max()
tempyear = int(tempyear)
templong = round(templong,4)

print(templat)
print(templong)
print(tempyear)
print(tempCNmax)      

25.4924
-80.5657
2010
8236


----

# Define Function to Pull DOY from Emissions Data:
### Function will go through Cluster Points file from DBScan and for each index row, will pull [Coordinates, ClusterNum and Year], and match these to the Emissions data file. Function will pull the DOY feature and place it into a new dataframe.

In [43]:
def pulldoyfromtargetdf(datapointsforcluster, targetdf_doy):
    clusterindex = 0
    clusterindex = int(clusterindex)
    
    cluserdatapoints_doy = pd.DataFrame()
    
    ### Test and execution While Loop.  It does work with 1 entry point. 
    #while clusterindex < 770256:
    while clusterindex < len(datapointsforcluster):
        ### for each coordinate point in the FireClusterNum sets which is the cluster group for each centerpoint from the DBScan files, go into the original data set and pull
        ### the doy over into a new dataframe. 
        tempclunum1 = datapointsforcluster.iloc[clusterindex]['ClusterNum']
        templat1 = datapointsforcluster.iloc[clusterindex]['0']
        templong1 = datapointsforcluster.iloc[clusterindex]['1']
        tempyear1 = datapointsforcluster.iloc[clusterindex]['Year']
        tempyear1 = int(tempyear1)
        
        search_doy1 = targetdf_doy.loc[(targetdf_doy['year'] == tempyear1) & (targetdf_doy['latitude'] == templat1) & (targetdf_doy['longitude'] == templong1)]
        ret_doy1 = search_doy1['doy']
        #retdoy1 = int(retdoy1)

        
        doytemp = pd.DataFrame({'latitude':[templat1], 'longitude': [templong1], 'ClusterNum':[tempclunum1], 'Year': [tempyear1], 'doy':[ret_doy1]})
        cluserdatapoints_doy = cluserdatapoints_doy.append(doytemp, ignore_index = True)

        clusterindex = clusterindex + 1     
    return cluserdatapoints_doy                                              
    

# Run Function to pull DOYs from target file (Emissions Data):

### Size of files:

In [44]:
a = len(Clusterpoints2010)
b = len(emdata2010)
print(a, b)

270486 378858


In [ ]:
start_time = time.time()
doypull_clusterpoints = pulldoyfromtargetdf(Clusterpoints2010, emdata2010)
message = 'run in {:,.2f} seconds'
print(message.format(time.time()-start_time))

In [ ]:
doypull_clusterpoints.tail(10)

In [ ]:
doypull_clusterpoints.to_csv('/Users/AlfHaugen/Python/Wildfire_Data/FireExports/ClusterDP_doy_2010.csv', encoding='utf-8')

# Define Function to determine the DOY Range for each Cluster Number:
### Function will go through each cluster number set and determing the Max and Min DOY for the cluster and pull into a new DF. 

In [ ]:
def determinedaterange(clusterdoydf):
    cnumindex = 0
    maxclusternum = clusterdoydf['ClusterNum'].max()
    ClusterNumDateRange = pd.DataFrame()
    
    while cnumindex < maxclusternum:
        clustnumsel = clusterdoydf[(clusterdoydf.ClusterNum == cnumindex)]
    
        doymax = clustnumsel['doy'].max()
        doymin = clustnumsel['doy'].min()
        doyyear = clustnumsel['Year']
    
        dftemp = ({'ClusterNum': [cnumindex], 'Year': [doyyear], 'doy_min': [doymin], 'doy_max': [doymax]})
        ClusterNumDateRange = ClusterNumDateRange.append(dftemp, ignore_index = True)
        
        cnumindex = cnumindex + 1
    
    return ClusterNumDateRange

------------------------

# ---  Alternative Did Not Work - only pulled one line --- 

# K-D Tree to Repeat Function to pull DOY from Emissions Data. 

### Define coordinates into cartesian numbers.  6371 is kms. 

In [13]:
def cartesian(latitude, longitude, elevation = 0):
    # Convert to radians
    latitude = latitude * (math.pi / 180)
    longitude = longitude * (math.pi / 180)

    R = 6371 # 6378137.0 + elevation  # relative to centre of the earth
    X = R * math.cos(latitude) * math.cos(longitude)
    Y = R * math.cos(latitude) * math.sin(longitude)
    Z = R * math.sin(latitude)
    return (X, Y, Z)

### Place target data/coordinates into a Places List in order to change into Cartesian coordinates. 

In [15]:
places = []
for index, row in emdata2010.iterrows():
    coordinates = [row['latitude'], row['longitude']]
    cartesian_coord = cartesian(*coordinates)
    places.append(cartesian_coord)

tree = spatial.KDTree(places)
places[1]

(940.8795796237181, -5674.6643169387, 2739.08223803334)

In [21]:
def find_coord_doy(lat, lon):
    cartesian_coord = cartesian(lat, lon)
    coordmatch = tree.query([cartesian_coord], p = 2)
    index = coordmatch[1][0]
    return coordmatch

## Function to determine match across coordinates and pull DOY over into new DF. 

def pulldoyfromtargetdf(datapointsforcluster, emdataset):
    clusterindex = 0
    clusterindex = int(clusterindex)
    
    cluserdatapoints_doy = pd.DataFrame()
    
    ### Test and execution While Loop.  It does work with 1 entry point. 
    while clusterindex < 5:
    #while clusterindex < len(datapointsforcluster):
        ### for each coordinate point in the FireClusterNum sets which is the cluster group for each centerpoint from the DBScan files, go into the original data set and pull
        ### the doy over into a new dataframe. 
        tempclunum2 = datapointsforcluster.iloc[clusterindex]['ClusterNum']
        templat2 = datapointsforcluster.iloc[clusterindex]['0']
        templong2 = datapointsforcluster.iloc[clusterindex]['1']
        tempyear2 = datapointsforcluster.iloc[clusterindex]['Year']
        tempyear2 = int(tempyear2)
        
        # Running K-D Tree function to find coordindate match:
        coord_match = find_coord_doy(templat2, templong2)
        targetlocation = coord_match[1]
        targetlocation = int(targetlocation)
        
        #Lines to pull data from the target dataframe, will need to be customized to the target DF. 
        target_lat = emdataset.iloc[targetlocation]['latitude']
        target_long = emdataset.iloc[targetlocation]['longitude']
        target_doy = emdataset.iloc[targetlocation]['doy']
        target_doy = int(target_doy)
        
        #Create new dataframe pulling in features from clusters and doy from original data set. 
        doytemp = pd.DataFrame({'cluster_latitude': [templat2], 'cluster_long':[templong2],'ClusterNum':[tempclunum2], 'target_latitude':[target_lat], 'target_longitude': [target_long],
                                'Year': [tempyear2], 'doy':[target_doy]})
        clusterdatapoints_doy = cluserdatapoints_doy.append(doytemp, ignore_index = True)

        clusterindex = clusterindex + 1     
    return clusterdatapoints_doy                                              
    

## Assess size of input files:

## Run Function

In [34]:
start_time = time.time()
cdoyfinal = pulldoyfromtargetdf(Clusterpoints2010, emdata2010)
message = 'run in {:,.2f} seconds'
print(message.format(time.time()-start_time))

run in 0.02 seconds


In [35]:
clusterdatapoints_doy

,cluster_latitude,cluster_long,ClusterNum,target_latitude,target_longitude,Year,doy
0,48.9655,-119.2912,8236.0,48.9655,-119.2912,2010,214


In [32]:
cdoyfinal.head(10)

,cluster_latitude,cluster_long,ClusterNum,target_latitude,target_longitude,Year,doy
0,48.9655,-119.2912,8236.0,48.9655,-119.2912,2010,214


In [30]:
Clusterpoints2010.iloc[5]

Unnamed: 0       5.0000
0               25.6299
1              -81.0530
ClusterNum       1.0000
Year          2010.0000
Name: 5, dtype: float64